In [3]:
import sys
sys.path.append('d:/flagellar/code/core')
sys.path.append('/flagellar/code/core/')
sys.path.append('/kaggle/input/my-flagellar-library/')
import flg_support as fls
import flg_unet
import flg_numerics
import flg_model
import importlib
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import flg_yolo

fast_mode = False
if fls.is_submission:
    fast_mode = False

In [4]:
data = fls.dill_load(fls.result_dir + '/many_full/Baseline_6_b12ad9a5_633 _f.pickle')

In [5]:
train_data = fls.load_all_train_data()[:5]

In [6]:
data.trained_model.run_in_parallel = False
inferred_data = data.trained_model.infer(train_data)
for d in inferred_data:
    print(d.labels_unfiltered)

C:\Users\jeroe\anaconda3\envs\jupyterlab-debugger2\lib\site-packages\cupyx\jit\_interface.py:173: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')


Clearing cupy
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
Processing tomogram tomo_003acc (1/1)
Processing 500 out of 500 slices (CONCENTRATION=1)
Processing 500 out of 500 slices (CONCENTRATION=1)
Processing 500 out of 500 slices (CONCENTRATION=1)
Processing 500 out of 500 slices (CONCENTRATION=1)
------------------
    index    z     y    x  confidence  i_model
2       5  119  1124  863    0.228149        0
3      20  143  1191  861    0.158203        0
5      39  144  1188  859    0.104858        0
6      42  238  1218  846    0.098083        0
7      51  238  1218  841    0.088806        0
8      52  120  1125  860    0.088196        0
9      54  238  1226  846    0.086609        0
10     59  119  1117 

In [ ]:
2